https://python.langchain.com/docs/use_cases/graph/integrations/graph_networkx_qa#create-the-graph

In [79]:
# import libraries
from langchain.indexes import GraphIndexCreator
from langchain.indexes.graph import NetworkxEntityGraph
from langchain_community.llms import Ollama
from langchain.chains import GraphQAChain
import networkx as nx
import pandas as pd

In [80]:
# read csv file and skip first row
highway_type = pd.read_csv('edge_labels_highways.csv', header=None, skiprows=1,
                 names=['edge', 'highway type'])
# split the first column into two columns
highway_type['edge'] = highway_type['edge'].apply(lambda x: eval(x))

# Split the 'edge' column into two columns 'start' and 'end'
highway_type[['startNode', 'endNode']] = highway_type['edge'].apply(pd.Series)
# convert the values into strings
highway_type['startNode'] = highway_type['startNode'].astype(str)
highway_type['endNode'] = highway_type['endNode'].astype(str)
# Drop the 'edge' column
highway_type.drop('edge', axis=1, inplace=True)
highway_type.head()

,highway type,startNode,endNode
0,footway,31404364,313157654
1,footway,31404364,7112240026
2,service,31404364,7120224692
3,NaN,7120224687,7112240050
4,NaN,7112240050,7112240049


In [81]:
# create networkx graph based on csv files
G = nx.read_edgelist("edges_UH_Graph.csv", delimiter=",", create_using=nx.DiGraph(), nodetype=str)
# add labels to edges according to highway
for u,v in G.edges():
    if (u,v) in zip(highway_type['startNode'], highway_type['endNode']):
        G[u][v]['relation'] = highway_type['highway type'].loc[(highway_type['startNode'] == u) & (highway_type['endNode'] == v)].values[0]
    else:
        G[u][v]['relation'] = 'unknown'

In [82]:
# write the graph to a GML file
nx.write_gml(G, "UH_Graph.gml")

In [83]:
# read the graph from the GML file
G = nx.read_gml("UH_Graph.gml")

In [84]:
# load gml graph as NetworkxEntityGraph
G = nx.read_gml("UH_Graph.gml")
G_NX = NetworkxEntityGraph(G)

In [85]:
# define the chain
chain = GraphQAChain.from_llm(Ollama(model="llama2"), graph = G_NX, verbose = False)

In [86]:
chain.run('Are the lables of the edges different?')